In [ ]:
这个是nn代码，可以直接运行，生成结果文件final_esim_fea.h5

In [1]:
from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss
import gc
from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np # linear algebra
import pandas as pd #

Using TensorFlow backend.


In [2]:
topic_embedding=np.load('esim_topic_embedding.npy')
word_embedding=np.load('esim_word_embedding.npy')

w_topic=np.load('w_topic.npy')
last_6_tpoic=np.load('last_6_tpoic.npy')
l_topic=np.load('l_topic.npy')
u_topic=np.load('u_topic.npy')
qtopic_10=np.load('qtopic_10.npy')
qtopic_50=np.load('qtopic_50.npy')

qtitlec_1=np.load('qtitlec_1.npy')
qtitlec=np.load('qtitlec.npy')
qinfoc=np.load('qinfoc.npy')

In [3]:
feature=np.load("all_data_final.npy")

In [4]:
feature_shape=feature.shape[1]

In [5]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
target=train['target']

In [6]:
import warnings
warnings.filterwarnings('ignore')
from keras.layers import *
from keras.initializers import *
from keras.layers import InputSpec, Layer, Input, Dense, merge, Conv1D
from keras.layers import Lambda, Activation, Dropout, Embedding, TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from keras.regularizers import l2
from keras.activations import softmax


def unchanged_shape(input_shape):
    "Function for Lambda layer"
    return input_shape
def substract(input_1, input_2):
    "Substract element-wise"
    neg_input_2 = Lambda(lambda x: -x, output_shape=unchanged_shape)(input_2)
    out_ = Add()([input_1, neg_input_2])
    return out_
def submult(input_1, input_2):
    "Get multiplication and subtraction then concatenate results"
    mult = Multiply()([input_1, input_2])
    sub = substract(input_1, input_2)
    out_= Concatenate()([sub, mult])
    return out_
def apply_multiple(input_, layers):
    "Apply layers to input then concatenate result"
    if not len(layers) > 1:
        raise ValueError('Layers list should contain more than 1 layer')
    else:
        agg_ = []
        for layer in layers:
            agg_.append(layer(input_))
        out_ = Concatenate()(agg_)
    return out_
def time_distributed(input_, layers):
    "Apply a list of layers in TimeDistributed mode"
    out_ = []
    node_ = input_
    for layer_ in layers:
        node_ = TimeDistributed(layer_)(node_)
    out_ = node_
    return out_
def soft_attention_alignment(input_1, input_2):
    "Align text representation with neural soft attention"
    attention = Dot(axes=-1)([input_1, input_2])
    w_att_1 = Lambda(lambda x: softmax(x, axis=1),
                     output_shape=unchanged_shape)(attention)
    w_att_2 = Permute((2,1))(Lambda(lambda x: softmax(x, axis=2),
                             output_shape=unchanged_shape)(attention))
    in1_aligned = Dot(axes=1)([w_att_1, input_1])
    in2_aligned = Dot(axes=1)([w_att_2, input_2])
    return in1_aligned, in2_aligned

In [7]:
def nnmodel(word_embedding=word_embedding,topic_embedding=topic_embedding):

###########################################q_topic l_topic match
    topic_embedding_50 = Embedding(
        input_dim=topic_embedding.shape[0],
        output_dim=topic_embedding.shape[1],
        weights=[topic_embedding],
        input_length=50,
        trainable=False
    ) 
    q_topic_50 = Input(shape=(50,), name="q_topic_50")
    l_topic    = Input(shape=(50,), name="l_topic")
    emb1 = topic_embedding_50(q_topic_50)
    emb2 = topic_embedding_50(l_topic)
    topic_encode = Bidirectional(CuDNNLSTM(64, return_sequences=True))
    t1_encoded = topic_encode(emb1)
    t2_encoded = topic_encode(emb2)
    t1_aligned, t2_aligned = soft_attention_alignment(t1_encoded, t2_encoded)
    t1_combined = Concatenate()([t1_encoded, t2_aligned, submult(t1_encoded, t2_aligned)])
    t2_combined = Concatenate()([t2_encoded, t1_aligned, submult(t2_encoded, t1_aligned)]) 
    topic_compose = Bidirectional(CuDNNLSTM(64, return_sequences=True))
    t1_compare = topic_compose(t1_combined)
    t2_compare = topic_compose(t2_combined)
    t1_rep = apply_multiple(t1_compare, [GlobalAvgPool1D(), GlobalMaxPool1D()])
    t2_rep = apply_multiple(t2_compare, [GlobalAvgPool1D(), GlobalMaxPool1D()])
###################################################################q_topic u_topic
    topic_embedding_10 = Embedding(
            input_dim=topic_embedding.shape[0],
            output_dim=topic_embedding.shape[1],
            weights=[topic_embedding],
            input_length=10,
            trainable=False
        ) 
    q_topic_10 = Input(shape=(10,), name="q_topic_10")
    u_topic = Input(shape=(10,), name="u_topic")
    u_topic_emb1 = topic_embedding_10(u_topic)
    q_topic_emb1 = topic_embedding_10(q_topic_10)
    topic_encode2 = Bidirectional(CuDNNLSTM(64, return_sequences=True))
    t1_encoded2 = topic_encode2(q_topic_emb1)
    t2_encoded2= topic_encode2(u_topic_emb1)
    t1_aligned2, t2_aligned2 = soft_attention_alignment(t1_encoded2, t2_encoded2)
    t1_combined2 = Concatenate()([t1_encoded2, t2_aligned2, submult(t1_encoded2, t2_aligned2)])
    t2_combined2 = Concatenate()([t2_encoded2, t1_aligned2, submult(t2_encoded2, t1_aligned2)]) 
    topic_compose2 = Bidirectional(CuDNNLSTM(64, return_sequences=True))
    t1_compare2 = topic_compose2(t1_combined2)
    t2_compare2 = topic_compose2(t2_combined2)
    t1_rep2 = apply_multiple(t1_compare2, [GlobalAvgPool1D(), GlobalMaxPool1D()])
    t2_rep2 = apply_multiple(t2_compare2, [GlobalAvgPool1D(), GlobalMaxPool1D()])
########################################################################## w_topic
    w_topic = Input(shape=(10,1), name="w_topic")
    u_topic_weight=Multiply()([u_topic_emb1, w_topic])
    
    conv1 = Conv1D(filters=128, kernel_size=1, padding='same', activation='relu')
    conv3 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')
    conv5 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')
    conv1a = conv1(u_topic_weight)
    conv1b = conv1(q_topic_emb1)
    glob1a = GlobalAveragePooling1D()(conv1a)
    glob1b = GlobalAveragePooling1D()(conv1b)
    conv3a = conv3(u_topic_weight)
    conv3b = conv3(q_topic_emb1)
    glob3a = GlobalAveragePooling1D()(conv3a)
    glob3b = GlobalAveragePooling1D()(conv3b)
    conv5a = conv5(u_topic_weight)
    conv5b = conv5(q_topic_emb1)
    glob5a = GlobalAveragePooling1D()(conv5a)
    glob5b = GlobalAveragePooling1D()(conv5b)
    mergea = concatenate([glob1a, glob3a,  glob5a])
    mergeb = concatenate([glob1b, glob3b,  glob5b])
    difftw = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=(2 * 128 + 1*32,))([mergea, mergeb])
    multw = Lambda(lambda x: x[0] * x[1], output_shape=(2 * 128 + 1*32,))([mergea, mergeb])
    wtmatch=concatenate([difftw,multw,mergea])
###########################################qtitlec  qtitlec_1 match
    qtitlec = Input(shape=(20,), name="qtitlec")
    qtitlec_1 = Input(shape=(20,), name="qtitlec_1")        
    title_word_embedding = Embedding(
        input_dim=word_embedding.shape[0],
        output_dim=word_embedding.shape[1],
        weights=[word_embedding],
        input_length=20,
        trainable=False
    )
    q1_embed = title_word_embedding(qtitlec)
    q2_embed = title_word_embedding(qtitlec_1)
    encode = Bidirectional(CuDNNLSTM(64, return_sequences=True))
    q1_encoded = encode(q1_embed)
    q2_encoded = encode(q2_embed)
    q1_aligned, q2_aligned = soft_attention_alignment(q1_encoded, q2_encoded)
    q1_combined = Concatenate()([q1_encoded, q2_aligned, submult(q1_encoded, q2_aligned)])
    q2_combined = Concatenate()([q2_encoded, q1_aligned, submult(q2_encoded, q1_aligned)]) 
    compose = Bidirectional(CuDNNLSTM(64, return_sequences=True))
    q1_compare = compose(q1_combined)
    q2_compare = compose(q2_combined)
    q1_rep = apply_multiple(q1_compare, [GlobalAvgPool1D(), GlobalMaxPool1D()])
    q2_rep = apply_multiple(q2_compare, [GlobalAvgPool1D(), GlobalMaxPool1D()])
#############################################################qinfoc
    qinfoc = Input(shape=(40,), name="qinfoc")
    qinfoc_word_embedding = Embedding(
        input_dim=word_embedding.shape[0],
        output_dim=word_embedding.shape[1],
        weights=[word_embedding],
        input_length=40,
        trainable=False
    )
    qinfocconv1 = Conv1D(filters=128, kernel_size=1, padding='same', activation='relu')
    qinfocconv3 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')
    qinfocconv5 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')
    qinfoc_embed = qinfoc_word_embedding(qinfoc)
    info_conv1a = conv1(qinfoc_embed)
    info_glob1a = GlobalAveragePooling1D()(info_conv1a)
    info_glob1b = GlobalMaxPool1D()(info_conv1a)
    info_conv3a = conv3(qinfoc_embed)
    info_glob3a = GlobalAveragePooling1D()(info_conv3a)
    info_glob3b = GlobalMaxPool1D()(info_conv3a)
    info_conv5a = conv5(qinfoc_embed)
    info_glob5a = GlobalAveragePooling1D()(info_conv5a)
    info_glob5b = GlobalMaxPool1D()(info_conv5a)
    info_merge = concatenate([info_glob1a, info_glob3a,info_glob5a,info_glob1b,info_glob3b,info_glob5b])
################################################feature
    feature = Input(shape=(feature_shape,), name="feature")
    fea1 = Dense(1024, activation='relu')(feature)
    dense = Dropout(0.3)(fea1)
    dense = Dense(512, activation='relu')(dense)
    dense = Dropout(0.3)(dense)
###############################################################
    last_6_tpoic = Input(shape=(50,), name="last_6_tpoic")
    emb1t = topic_embedding_50(q_topic_50)
    emb2t = topic_embedding_50(last_6_tpoic)
    qtcconv1 = Conv1D(filters=128, kernel_size=1, padding='same', activation='relu')
    qtcconv2 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')
    qtcconv3 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')
    conv1at = qtcconv1(emb1t)
    conv1bt = qtcconv1(emb2t)
    glob1at = GlobalAveragePooling1D()(conv1at)
    glob1bt = GlobalAveragePooling1D()(conv1bt)
    conv2at = qtcconv2(emb1t)
    conv2bt = qtcconv2(emb2t)
    glob2at = GlobalAveragePooling1D()(conv2at)
    glob2bt = GlobalAveragePooling1D()(conv2bt)
    conv3at = qtcconv3(emb1t)
    conv3bt = qtcconv3(emb2t)
    glob3at = GlobalAveragePooling1D()(conv3at)
    glob3bt = GlobalAveragePooling1D()(conv3bt)
    mergeat = concatenate([glob1at, glob2at,  glob3at])
    mergebt = concatenate([glob1bt, glob2bt,  glob3bt])
    difft = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=(2 * 128 + 1*32,))([mergeat, mergebt])
    mult = Lambda(lambda x: x[0] * x[1], output_shape=(2 * 128 + 1*32,))([mergeat, mergebt])
###########################################main layer
    
    
    merge =concatenate([q1_rep,q2_rep,t1_rep,t2_rep,wtmatch,info_merge,t1_rep2,t2_rep2,difft,mult,dense])#diff,mul])#,categ,,dense
    main_l = Dense(1024, activation='relu', )(merge)
    main_l = Dropout(0.3)(main_l)
    main_l = Dense(512, activation='relu', )(main_l)
    
    output = Dense(1,activation='sigmoid') (main_l)

    model = Model(inputs=[qtitlec,qtitlec_1,q_topic_10,l_topic,u_topic,w_topic,qinfoc,q_topic_50,last_6_tpoic,feature], outputs=output)#feature

    model.compile(optimizer ='adam',
                  loss= 'binary_crossentropy',
                  metrics=['acc'])
    return model

In [8]:
def make_data(index_):
    data=dict()
    data['w_topic']=w_topic_[index_]
    data['last_6_tpoic']=last_6_tpoic[index_]
    data['q_topic_10']=qtopic_10[index_]
    data['q_topic_50']=qtopic_50[index_]
    data['qtitlec_1']=qtitlec_1[index_]
    data['qtitlec']=qtitlec[index_]
    data['qinfoc']=qinfoc[index_]
    data['feature']=feature[index_]
    data['l_topic']=l_topic[index_]
    data['u_topic']=u_topic[index_]
    return data

In [9]:
w_topic_=[]
for i in range(len(w_topic)):
    w_topic_.append(w_topic[i].reshape(-1,1))
w_topic_=np.array(w_topic_)
del w_topic

In [10]:
sub1 = np.zeros((test.shape[0],1 ))
oof_pref1 = np.zeros((train.shape[0], 1))
count=0

In [11]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics

skf = StratifiedKFold(n_splits=5, random_state=1996, shuffle=True)
for index, (train_index, test_index) in enumerate(skf.split(train, target)):
    K.clear_session()
    filepath = "esim_word_nofea_%d.h5" % count
    checkpoint = ModelCheckpoint(
        filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', factor=0.8, patience=1, min_lr=0.0001, verbose=1)
    earlystopping = EarlyStopping(
        monitor='val_loss', min_delta=0.0001, patience=1,verbose=1, mode='auto')
    callbacks = [checkpoint, reduce_lr, earlystopping]
    
    model = nnmodel()
    print(index)
    train_x=make_data(train_index)
    test_x=make_data(test_index)
    train_y, test_y =target[train_index], target[test_index]
    print('start')
    history = model.fit(train_x,train_y, batch_size=2048, epochs=10, verbose=1, validation_data=(test_x, test_y),callbacks=callbacks,)
    model.load_weights(filepath)
    sub1 += model.predict(make_data(range(len(train),len(train)+len(test))), batch_size=4000)
    oof_pref1[test_index] = model.predict(test_x, batch_size=4000)
    count += 1
    print('val_auc:',metrics.roc_auc_score(test_y,oof_pref1[test_index]))#0.42480

W1217 17:23:10.740691 139812632864512 deprecation_wrapper.py:119] From /root/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:92: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W1217 17:23:10.742725 139812632864512 deprecation_wrapper.py:119] From /root/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1217 17:23:10.766636 139812632864512 deprecation_wrapper.py:119] From /root/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:99: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1217 17:23:10.768985 139812632864512 deprecation_wrapper.py:119] From /root/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeho

0
start
Train on 7591329 samples, validate on 1897833 samples
Epoch 1/10
7591329/7591329 [==============================] - 1099s 145us/step - loss: 0.3572 - acc: 0.8457 - val_loss: 0.3318 - val_acc: 0.8558

Epoch 00001: val_loss improved from inf to 0.33176, saving model to esim_word_nofea_0.h5
Epoch 2/10
7591329/7591329 [==============================] - 1074s 141us/step - loss: 0.3291 - acc: 0.8565 - val_loss: 0.3213 - val_acc: 0.8605

Epoch 00002: val_loss improved from 0.33176 to 0.32125, saving model to esim_word_nofea_0.h5
Epoch 3/10
7591329/7591329 [==============================] - 1077s 142us/step - loss: 0.3205 - acc: 0.8604 - val_loss: 0.3203 - val_acc: 0.8615

Epoch 00003: val_loss improved from 0.32125 to 0.32032, saving model to esim_word_nofea_0.h5
Epoch 4/10
7591329/7591329 [==============================] - 1059s 139us/step - loss: 0.3147 - acc: 0.8628 - val_loss: 0.3158 - val_acc: 0.8628

Epoch 00004: val_loss improved from 0.32032 to 0.31577, saving model to esim_wo

In [ ]:
sub2 = np.zeros((test.shape[0],1 ))
oof_pref2 = np.zeros((train.shape[0], 1))
count=0

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics

skf = StratifiedKFold(n_splits=5, random_state=1996, shuffle=True)
for index, (train_index, test_index) in enumerate(skf.split(train, target)):
    K.clear_session()
    filepath = "esim_word_nofea_%d.h5" % index
    model = nnmodel()
    print(index)
    train_x=make_data(train_index)
    test_x=make_data(test_index)
    train_y, test_y =target[train_index], target[test_index]
    print('start')
    model.load_weights(filepath)
    sub2 += model.predict(make_data(range(len(train),len(train)+len(test))), batch_size=4000)
    oof_pref2[test_index] = model.predict(test_x, batch_size=4000)
    count += 1
    print('val_auc:',metrics.roc_auc_score(test_y,oof_pref2[test_index]))

In [12]:
stacking=pd.concat([pd.DataFrame(oof_pref1),pd.DataFrame(sub1/5)])
stacking.columns=['esim_fea']

In [14]:
stacking=stacking.reset_index(drop=True)
stacking.to_hdf('final_esim_fea.h5', key='data')

In [17]:
sub1/=5
sub1.mean()

0.034193715641305855